# Infer.NET - Murder Mystery example
A simple example of inference. By Andy Gordon & Infer.NET colleagues at Microsoft Research.

Prior assumptions:  
* Either Alice or Bob dunnit, using either a gun or a pipe  
* Alice dunnit 30%, Bob dunnit 70%  
* Alice uses gun 3%, uses pipe 97%  
* Bob uses gun 80%, uses pipe 20%  

### Links
https://www.microsoft.com/en-us/research/wp-content/uploads/2016/02/fun-probabilistic_programming_obt_january_2013.pdf
https://github.com/prgoodwin/HabilisX/blob/master/Infer.NET%202.5/Samples/Fun/MurderMystery/MurderMystery.fs
http://infernet.azurewebsites.net/docs/default.aspx

## Load & open packages

### From zip

This solution due to Tomas Petricek.

https://www.microsoft.com/en-us/download/confirmation.aspx?id=52476   
Infer.NET 2.7.1803.1901.zip

https://download.microsoft.com/download/D/C/4/DC44E915-26FE-48AB-BD7C-AC1D92B23230/Infer.NET%202.7.1803.1901.zip

In [1]:
let wc = new System.Net.WebClient()
wc.DownloadFile(
    "https://download.microsoft.com/download/D/C/4/DC44E915-26FE-48AB-BD7C-AC1D92B23230/Infer.NET%202.7.1803.1901.zip", 
    "/home/nbuser/IfSharp/bin/packages/infer.zip")

Unzip it (could get .NET `ZipFile` to do this, but invoking Linux command line tools works)

In [7]:
open System.Diagnostics
let ps = 
    ProcessStartInfo(
        FileName="unzip", 
        Arguments="/home/nbuser/IfSharp/bin/packages/infer.zip -d /home/nbuser/IfSharp/bin/packages/infer",
        RedirectStandardOutput=true,UseShellExecute=false)
let p = Process.Start(ps)
p.StandardOutput.ReadToEnd()

"牁档癩㩥†栯浯⽥扮獵牥䤯卦慨灲戯湩瀯捡慫敧⽳湩敦⹲楺ੰ"

Now you can reference the extracted DLL:

In [3]:
#r "/home/nbuser/IfSharp/bin/packages/infer/Bin/Infer.FSharp.dll"

## Common code

The script **Packet.fsx** is created in the notebook "**Create paket script**"

Run this only once for each library.

In [4]:
// This is the official best practice using packet, however, the nuget package is not being maintaine
//#load "Paket.fsx"
//Paket.Package ["Infer.NET"]

In [5]:
open System
open MicrosoftResearch.Infer
open MicrosoftResearch.Infer.FSharp

In [6]:
[<ReflectedDefinition>]
let mystery (): bool*bool =
  let aliceDunnit = random (Bernoulli 0.30)
  let withGun = if aliceDunnit then random (Bernoulli 0.03) else random (Bernoulli 0.80)
  aliceDunnit, withGun

The value or constructor 'random' is not defined. Maybe you want one of the following:
   Random
The value or constructor 'random' is not defined. Maybe you want one of the following:
   Random
The value or constructor 'random' is not defined. Maybe you want one of the following:
   Random

In [11]:
open MicrosoftResearch.Infer.Distributions

The namespace 'Distributions' is not defined.

In [ ]:
let (aliceDunnitPrior: IDistribution<bool>, 
     withGunPrior: IDistribution<bool>) = infer <@ mystery @> ()

In [ ]:
// Our prior suggests Alice didn't do it, hence Bob dunnit, most likely with a gun.

In [ ]:
printfn "aliceDunnitPrior: %O" aliceDunnitPrior
printfn "withGunPrior: %O" withGunPrior

In [ ]:
[<ReflectedDefinition>]
let GunFoundAtScene (gunFound:bool): bool =
  let aliceDunnit, withGun = mystery () 
  observe(withGun = gunFound)
  aliceDunnit

In [ ]:
// Well, we found no gun (but a pipe) at the scene

In [ ]:
let posterior: IDistribution<bool> = infer <@ GunFoundAtScene @> false 

In [ ]:
// Given the observed pipe, the posterior suggests Alice dunnit!

In [ ]:
printfn "aliceDunnitPosterior: %O" posterior